In [1]:
import socket
import threading
import time

# Thread lock for printing results without overlap when using threads
print_lock = threading.Lock()

# List to store scan results for later saving
results_list = []

# Function to retrieve the banner from an open port (if available)
def get_banner(host_ip, port_number):
    try:
        s = socket.socket()
        s.settimeout(1)
        s.connect((host_ip, port_number))
        banner = s.recv(1024).decode().strip()
        s.close()
        return banner
    except:
        return None

# Function to map port number to common service name
def get_service(port_number):
    try:
        return socket.getservbyport(port_number)
    except:
        return "Unknown"

# Function to scan a single port and collect its info
def scan_single_port(host_ip, port_number):
    try:
        s = socket.socket()
        s.settimeout(0.5)
        connection_result = s.connect_ex((host_ip, port_number))

        if connection_result == 0:
            service_name = get_service(port_number)
            banner = get_banner(host_ip, port_number)

            # Save results to the list for file writing
            results_list.append(f"Port {port_number} | Service: {service_name} | Banner: {banner or 'N/A'}")

            # Print open port info with lock to avoid print overlap
            with print_lock:
                print(f"[+] Port {port_number} is OPEN - Service: {service_name}")
                if banner:
                    print(f"    Banner: {banner}")
        s.close()
    except:
        pass

# Function to scan all ports in the given range using threading
def scan_port_range(host_ip, port_start, port_end):
    threads = []
    for port_number in range(port_start, port_end + 1):
        thread = threading.Thread(target=scan_single_port, args=(host_ip, port_number))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

# Function to save scan results to a file
def save_results(filename):
    try:
        with open(filename, 'w') as file:
            file.write("Port Scanner Results\n")
            file.write("====================\n")
            if not results_list:
                file.write("No open ports found.\n")
            else:
                for entry in results_list:
                    file.write(entry + "\n")
        print(f"\n Results saved to '{filename}'")
    except Exception as error:
        print(f"Error saving results: {error}")

# Main execution block
if __name__ == "__main__":
    # Get target IP or hostname from user
    target_input = input("Enter IP address or hostname to scan: ").strip()
    try:
        resolved_ip = socket.gethostbyname(target_input)
    except socket.gaierror:
        print("Invalid hostname.")
        exit()

    # Get port range from user
    try:
        port_start = int(input("Start port: "))
        port_end = int(input("End port: "))
        if port_start < 1 or port_end > 65535 or port_start > port_end:
            print("Invalid port range.")
            exit()
    except ValueError:
        print("Invalid input. Ports must be integers.")
        exit()

    # Start scanning
    print(f"\n Starting scan on {target_input} [{resolved_ip}] from port {port_start} to {port_end}...\n")
    start_time = time.time()
    scan_port_range(resolved_ip, port_start, port_end)
    print(f"\n Scan completed in {round(time.time() - start_time, 2)} seconds.")

    # Prompt to save results
    save_prompt = input("Do you want to save the results to a file? (y/n): ").lower()
    if save_prompt == 'y':
        file_name = input("Enter filename to save (e.g., results.txt): ").strip()
        save_results(file_name)


Enter IP address or hostname to scan:  192.168.1.1
Start port:  0
End port:  0


Invalid port range.

 Starting scan on 192.168.1.1 [192.168.1.1] from port 0 to 0...


 Scan completed in 0.0 seconds.


Do you want to save the results to a file? (y/n):  n
